In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVR
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import TransformerMixin

In [2]:
hourly=pd.read_csv('./hour.csv')
daily=pd.read_csv('./hour.csv')

In [3]:
hourly.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
daily.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [41]:
#X1_Train = hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','registered','dteday'],axis=1) ## CASUAL IN DATA
#X2_Train =  hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','casual','dteday'],axis=1) ## REGISTERED IN DATA 
#X3_Train = hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA

#X1_Test = hourly[hourly['dteday'] >= '2012-10-01'].drop(['cnt','registered','dteday'],axis=1)## CASUAL IN DATA
#X2_Test =  hourly[hourly['dteday'] >='2012-10-01'].drop(['cnt','casual','dteday'],axis=1)## REGISTERED IN DATA
#X3_Test = hourly[hourly['dteday'] >= '2012-10-01'].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA

#y_cas_train =hourly['casual'][hourly['dteday'] < '2012-10-01']
#y_cnt_train =hourly['cnt'][hourly['dteday'] < '2012-10-01']
#y_reg_train =hourly['registered'][hourly['dteday'] < '2012-10-01']

#y_cas_test =hourly['casual'][hourly['dteday'] >= '2012-10-01']
#y_cnt_test =hourly['cnt'][hourly['dteday'] >= '2012-10-01']
#y_reg_test =hourly['registered'][hourly['dteday'] >= '2012-10-01']

In [7]:
#print(y_cnt_test)
#print(y_reg_test + y_pred)
#print(np.ravel(y_cnt_test),np.ravel((y_reg_test + y_pred)))

In [8]:
#SVR_Lin = SVR(kernel='linear',C=1e3)
#SVR_Lin.fit(X2_Train, y_cas_train)
#y_pred=SVR_Lin.predict(X2_Test)
#SVR_Lin.score(np.ravel(y_cnt_test),np.ravel((y_reg_test + y_pred)))

In [9]:
#regr_2 = AdaBoostRegressor(regr.fit(X2_Train, y_cas_train))
#regr_2.fit(X2_Train, y_cas_train)
#y_2 = regr_2.predict(X2_Test)


In [10]:
#regr_2.estimator_errors_ 

In [8]:
# Create linear regression object
regr = LinearRegression(fit_intercept =True)
regr.fit(X_Train, Y_cnt_train) # Using Registered & Using Casual
y_pred = regr.predict(X_Test) # Using Registered for predicting
regr.score(X_Test, Y_cnt_test) # R2 score on Casual + Registered = Total 

1.0

In [10]:
polynomial_features=PolynomialFeatures(degree=2)
x_poly=polynomial_features.fit_transform(X_Train)
X_test_poly=polynomial_features.fit_transform(X_Test)
model=LinearRegression()
model.fit(x_poly, Y_cnt_train)
y_poly_pred = model.predict(X_test_poly)
model.score(X_test_poly, Y_cnt_test)

1.0

In [12]:
dataset = pd.DataFrame({'y_poly_pred':y_poly_pred,'Y_pred':y_pred,'actual':Y_cnt_test})
dataset.head()

,y_poly_pred,Y_pred,actual
15211,45.0,45.0,45
15212,18.0,18.0,18
15213,12.0,12.0,12
15214,7.0,7.0,7
15215,10.0,10.0,10


In [13]:
stacker= LinearRegression()
stacker.fit(dataset[['y_poly_pred', 'Y_pred']], dataset['actual'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [14]:
temp=dataset[['y_poly_pred','Y_pred']]

In [15]:
tar=dataset[['actual']]

In [16]:
prediction=stacker.predict(temp)

In [17]:
r2_score(tar,prediction)

1.0

In [22]:
date =hourly['dteday'][hourly['dteday'] >= '2012-10-01']
date=pd.DataFrame(date)
date.reset_index(drop=True,inplace=True)


In [37]:
X_Train = hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','dteday'],axis=1)## NONE OF THst_EM IN DATA
X_Test = hourly[hourly['dteday'] >= '2012-10-01'].drop(['cnt','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test =hourly['cnt'][hourly['dteday'] >= '2012-10-01']
Y_cnt_train =hourly['cnt'][hourly['dteday'] < '2012-10-01']

In [19]:
from sklearn.ensemble import GradientBoostingRegressor
model= GradientBoostingRegressor()
model.fit(X_Train, Y_cnt_train)
model.score(X_Test, Y_cnt_test)

0.9992730727681748

In [31]:
temp.reset_index(drop=True,inplace=True)
#y_poly_pred=pd.DataFrame(y_poly_pred)
#y_poly_pred.reset_index(drop=True,inplace=True)
#one=pd.DataFrame(model.predict(X_Test))
two=pd.DataFrame(Y_cnt_test)
one.reset_index(drop=True,inplace=True)
two.reset_index(drop=True,inplace=True)
new_dataset=pd.concat([date,temp,two],axis=1)

In [38]:
X_Train = new_dataset[new_dataset['dteday'] < '2012-11-01'].drop(['cnt','dteday'],axis=1)## NONE OF THst_EM IN DATA
X_Test = new_dataset[new_dataset['dteday'] >= '2012-11-01'].drop(['cnt','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test =new_dataset['cnt'][new_dataset['dteday'] >= '2012-11-01']
Y_cnt_train =new_dataset['cnt'][new_dataset['dteday'] < '2012-11-01']

In [40]:
model= GradientBoostingRegressor()
model.fit(X_Train, Y_cnt_train)
model.score(X_Test, Y_cnt_test)

0.9999116919406402